# Setup

On google colab, you have to restart runtime after running the following line

In [2]:
!pip install omegaconf

In [3]:
from google.colab import drive
drive.mount("/content/drive/")
#"/content/drive/My Drive/NN-kNN/"
folder_name = "/content/drive/Othercomputers/My MacBook Pro/GitHub/NN-kNN/"
import sys
sys.path.insert(0,folder_name)

Mounted at /content/drive/


In [4]:
##This is added because my Rdata uses Cdata for the covid data set.
##Rdata use Cdata function to load the data set, then convert it to regression problem
import os
import sys
sys.path.append('/content/drive/Othercomputers/My MacBook Pro/GitHub/NN-kNN/dataset')


In [5]:
# folder_name = os.getcwd()

In [6]:
import torch
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
from omegaconf import DictConfig, OmegaConf

from dataset import cls_small_data as Cdata
import model.cls_model as Cmodel
from dataset import cls_medium_data

from dataset import reg_data as Rdata
import model.reg_model as Rmodel

In [7]:
conf_file = OmegaConf.load(os.path.join(folder_name, 'config.yaml'))

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
torch.cuda.is_available()

True

# NCA and LMNN setup

In [10]:
pip install metric-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 3.0 MB/s eta 0:00:00


In [11]:
import metric_learn
from metric_learn import LMNN,NCA

# Data Sets

Supported small dataset for classification:  
'zebra',
'zebra_special',
'bal',
'digits',
'iris',
'wine',
'breast_cancer',

for regression:
'califonia_housing',
'abalone',
'diabets',
'body_fat',
'ziweifaces'


Newly added data sets for mental health (psychology):

Classification:
'psych_depression_physical_symptons',
'covid_anxious',
'covid_depressed'


In [33]:
dataset_name = 'covid_depressed'
cfg = conf_file['dataset'][dataset_name]
#TODO need to add other covid data sets here.
if dataset_name in ['covid_depressed','covid_anxious','covid_physical','covid_lonely','covid_hopeless',
                    'psych_depression_physical_symptons',
                    'zebra','zebra_special','bal','digits','iris','wine','breast_cancer']:
    criterion = torch.nn.CrossEntropyLoss()
    Xs, ys = Cdata.Cls_small_data(dataset_name)
elif dataset_name in []:
    criterion = torch.nn.CrossEntropyLoss()
    Xs, ys = cls_medium_data.Cls_medium_data(dataset_name)
else:
    criterion = torch.nn.MSELoss()
    Xs, ys = Rdata.Reg_data(dataset_name)

Columns in the dataset: Index(['SU_ID', 'P_PANEL', 'NATIONAL_WEIGHT', 'REGION_WEIGHT',
       'NATIONAL_WEIGHT_POP', 'REGION_WEIGHT_POP', 'NAT_WGT_COMB_POP',
       'REG_WGT_COMB_POP', 'P_GEO', 'SOC1',
       ...
       'REGION9', 'P_DENSE', 'MODE', 'LANGUAGE', 'MAIL50', 'RACE1_BANNER',
       'RACE2_BANNER', 'INC_BANNER', 'AGE_BANNER', 'HH_BANNER'],
      dtype='object', length=177)
   SOC1  SOC2A  SOC2B  SOC3A  SOC3B  SOC4A  SOC4B  PHYS8  PHYS1A  PHYS1B  \
0   3.0    4.0    4.0    1.0    1.0    2.0    2.0    3.0     1.0     1.0   
1   3.0    3.0    3.0    1.0    3.0    2.0    2.0    2.0     2.0     2.0   
2   3.0    3.0    3.0    1.0    1.0    2.0    1.0    2.0     2.0     1.0   
3   3.0    2.0    2.0    1.0    1.0    2.0    1.0    3.0     2.0     2.0   
4   3.0    1.0    1.0    1.0    1.0    2.0    2.0    3.0     1.0     1.0   
5   2.0    3.0    3.0    1.0    1.0    1.0    1.0    2.0     2.0     2.0   
6   2.0    4.0    3.0    1.0    1.0    2.0    2.0    NaN     1.0     2.0   
7   3

In [34]:
# This section is used to reload the imported module.
# For example, if you made any changes in the model.cls_model, you should run importlib.reload(Cmodel) as long as you set import model.cls_model as Cmodel.
import importlib
importlib.reload(Rdata)
importlib.reload(Cdata)

<module 'dataset.cls_small_data' from '/content/drive/Othercomputers/My MacBook Pro/GitHub/NN-kNN/dataset/cls_small_data.py'>

# Classification with NNKNN

In [35]:
# prompt: get the unique y values and their counts

unique_values, counts = np.unique(ys, return_counts=True)
print(f"Unique values: {unique_values}")
print(f"Counts: {counts}")
print(f"Xs.size(): {Xs.size()}")


Unique values: [0 1 2 3]
Counts: [776 776 776 776]
Xs.size(): torch.Size([3104, 161])


In [36]:
def train_cls(X_train,y_train, X_test, y_test, cfg:DictConfig):
  X_train = X_train.to(device)
  y_train = y_train.to(device)
  X_test = X_test.to(device)

  train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train, y_train), batch_size=cfg.batch_size, shuffle=True)

  # Train model
  model = Cmodel.NN_k_NN(X_train,
                         y_train,
                         cfg.ca_weight_sharing,
                         cfg.top_case_enabled,
                         cfg.top_k,
                         cfg.discount,
                         device=device)

  optimizer = torch.optim.Adam(model.parameters(), lr=cfg.learning_rate) #, weight_decay=1e-5)

  patience_counter = 0
  for epoch in range(cfg.training_epochs):
    epoch_msg = True

    for X_train_batch, y_train_batch in train_loader:
      model.train()
      _, _, output, predicted_class = model(X_train_batch)
      loss = criterion(output, y_train_batch)

      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      if epoch_msg and (epoch + 1) % 2 == 0:
        print(f'Epoch [{epoch + 1}/{cfg.training_epochs}], Loss: {loss.item():.4f}')

        epoch_msg = False
      # print("evaluating")
    model.eval()
    with torch.no_grad():
      _, _, output, predicted_class = model(X_test)

      # Calculate accuracy
      accuracy_temp = accuracy_score(y_test.numpy(), predicted_class.cpu().numpy())
    if epoch == 0:
      best_accuracy = accuracy_temp
      torch.save(model.state_dict(), cfg.PATH)

    elif accuracy_temp > best_accuracy:
      #memorize best model
      torch.save(model.state_dict(), cfg.PATH)
      best_accuracy = accuracy_temp
      patience_counter = 0

    elif patience_counter > cfg.patience:
      model.eval()
      print("patience exceeded, loading best model")
      break
    else:
      patience_counter += 1

  return best_accuracy, model

In [37]:
def load_model_cls(X_train,y_train,cfg):
  # Define the model architecture
  model = Cmodel.NN_k_NN(
      X_train,
      y_train,
      cfg.ca_weight_sharing,
      cfg.top_case_enabled,
      cfg.top_k,
      cfg.discount,
      device=device
  )
  # Load the state dictionary
  model.load_state_dict(torch.load(cfg.path))
  model.to(device)
  model.eval()
  return model

In [ ]:
accuracies = []
knn_accuracies = []
lmnn_accuracies = []
PATH = os.path.join(folder_name, f'checkpoints/classifier_{dataset_name}.h5')
cfg.PATH = PATH
k_fold = KFold(n_splits=10, shuffle=True, random_state = None)
enable_lmnn = True

for train_index, test_index in k_fold.split(Xs):
  # Get training and testing data
  X_train, X_test = Xs[train_index], Xs[test_index]
  y_train, y_test = ys[train_index], ys[test_index]
  if(enable_lmnn):
    # https://contrib.scikit-learn.org/metric-learn/supervised.html#lmnn
    lmnn = LMNN(n_neighbors=5, learn_rate=1e-6)
    ##TODO, change here if you need to use a different one
    # lmnn = metric_learn.MLKR()
    # lmnn = metric_learn.NCA(max_iter=1000)
    lmnn.fit(X_train,y_train)
    knn = KNeighborsClassifier(n_neighbors=5,metric=lmnn.get_metric())
    knn.fit(X_train,y_train)
    # klmnn_accuracies.append( accuracy_score(knn.predict(X_test), y_test))
    lmnn_acc = accuracy_score(knn.predict(X_test), y_test)
    lmnn_accuracies.append(lmnn_acc)
    # continue

  knn =  KNeighborsClassifier(n_neighbors=cfg.top_k)
  knn.fit(X_train, y_train)
  knn_acc  = accuracy_score(knn.predict(X_test), y_test)
  knn_accuracies.append(knn_acc)

  best_accuracy, model = train_cls(X_train,y_train, X_test, y_test, cfg)
  accuracies.append(best_accuracy)
  break

print(f"Average accuracy:{np.mean(accuracies):.3f}")
print(f"KNN accuracy:{np.mean(knn_accuracies):.3f}")
print(f"LMNN/NCA accuracy:{np.mean(lmnn_accuracies):.3f}")


In [ ]:
print(f"Average accuracy:{np.mean(accuracies):.3f}")
print(f"KNN accuracy:{np.mean(knn_accuracies):.3f}")
print(f"LMNN/NCA accuracy:{np.mean(lmnn_accuracies):.3f}")

# Regression with NNKNN

In [24]:
unique_values, counts = np.unique(ys, return_counts=True)
print(f"Unique values: {unique_values}")
print(f"Counts: {counts}")
print(f"Xs.size(): {Xs.size()}")

Unique values: [-1.3414135  -0.44713783  0.44713783  1.3414135 ]
Counts: [738 738 738 738]
Xs.size(): torch.Size([2952, 161])


In [25]:
def train_reg(X_train,y_train, X_test, y_test, cfg:DictConfig):
  X_train = X_train.to(device)
  y_train = y_train.to(device)
  X_test = X_test.to(device)

  train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train, y_train), batch_size=cfg.batch_size, shuffle=True)
  test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_test, y_test), batch_size=cfg.batch_size, shuffle=False)


    # Train model
  model = Rmodel.NN_k_NN_regression(X_train,
                                    y_train,
                                    cfg.ca_weight_sharing,
                                    cfg.top_case_enabled,
                                    cfg.top_k,
                                    cfg.discount,
                                    cfg.class_weight_sharing,
                                    device=device)

  optimizer = torch.optim.Adam(model.parameters(), lr=cfg.learning_rate) #, weight_decay=1e-5)

  patience_counter = 0
  for epoch in range(cfg.training_epochs):
    # break # no training
    epoch_msg = True
    for X_train_batch, y_train_batch in train_loader:
      model.train()
      _, _, _, predicted_number = model(X_train_batch)
      # break
      loss = criterion(predicted_number.squeeze(), y_train_batch)
      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      if epoch_msg and (epoch + 1) % 2 == 0:
        epoch_msg = False
        print(f'Epoch [{epoch + 1}/{cfg.training_epochs}], Loss: {loss.item():.4f}')

    model.eval()
    with torch.no_grad():
      predicted_numbers = []
      for X_test_batch, _ in test_loader:
        X_test_batch = X_test_batch.to(device)
        _, _, _, predicted_number = model(X_test_batch)
        predicted_numbers.extend(predicted_number.squeeze().cpu().detach())

      predicted_numbers = torch.Tensor(predicted_numbers)
      accuracy_temp = criterion(y_test, predicted_numbers)

    if epoch == 0:
      best_accuracy = accuracy_temp
      torch.save(model.state_dict(), cfg.PATH)
    elif accuracy_temp < best_accuracy:
      torch.save(model.state_dict(), cfg.PATH)
      best_accuracy = accuracy_temp
      patience_counter = 0
    elif patience_counter > cfg.patience:
      model.eval()
      print("patience exceeded, loading best model")
      break
    else:
      patience_counter += 1

  _, case_activations, _, predicted_number = model(X_test)

  top_case_indices = torch.topk(case_activations, 5, dim=1)[1].cpu()

  accuracy = criterion(y_test, predicted_number.squeeze().cpu())
  y_train = y_train.cpu()
  top_k_average_accuracy = mean_squared_error(torch.mean(y_train[top_case_indices], dim=1), y_test)

  return best_accuracy, accuracy, top_k_average_accuracy, model

In [26]:
# prompt: load_model_reg()

def load_model_reg(X_train,y_train,cfg):
  # Define the model architecture
  model = Rmodel.NN_k_NN_regression(
      X_train,
      y_train,
      cfg.ca_weight_sharing,
      cfg.top_case_enabled,
      cfg.top_k,
      cfg.discount,
      cfg.class_weight_sharing,
      device=device
  )
  # Load the state dictionary
  model.load_state_dict(torch.load(cfg.path))
  model.to(device)
  model.eval()
  return model


In [28]:
best_accuracies = []
accuracies = []
top_k_average_accuracies = []
knn_accuracies = []
PATH = os.path.join(folder_name, f'checkpoints/regression_{dataset_name}.h5')
cfg.PATH = PATH
k_fold = KFold(n_splits=10, shuffle = True,random_state = None)


for train_index, test_index in k_fold.split(Xs):
  # Get training and testing data
  X_train, X_test = Xs[train_index], Xs[test_index]
  y_train, y_test = ys[train_index], ys[test_index]

  knn = KNeighborsRegressor(n_neighbors=cfg.top_k)
  knn.fit(X_train, y_train)
  knn_accuracies.append(mean_squared_error(knn.predict(X_test), y_test))

  best_accuracy, accuracy, top_k_average_accuracy, model= train_reg(X_train, y_train, X_test, y_test, cfg)
  best_accuracies.append(best_accuracy)
  accuracies.append(accuracy)
  top_k_average_accuracies.append(top_k_average_accuracy)
  # break

print("Average accuracy:", np.mean([acc.detach().numpy() for acc in accuracies]))
print("Average top_k_average_accuracies", np.mean(top_k_average_accuracies))
print("KNN accuracy:", np.mean(knn_accuracies))

Epoch [2/1000], Loss: 0.8961
Epoch [4/1000], Loss: 0.3790
Epoch [6/1000], Loss: 0.6677
Epoch [8/1000], Loss: 0.5964
Epoch [10/1000], Loss: 0.4243
Epoch [12/1000], Loss: 0.3253
Epoch [14/1000], Loss: 0.2069
Epoch [16/1000], Loss: 0.4395
Epoch [18/1000], Loss: 0.3927
Epoch [20/1000], Loss: 0.3533
Epoch [22/1000], Loss: 0.3096
Epoch [24/1000], Loss: 0.2666
Epoch [26/1000], Loss: 0.2718
Epoch [28/1000], Loss: 0.3268
Epoch [30/1000], Loss: 0.3962
Epoch [32/1000], Loss: 0.4162
Epoch [34/1000], Loss: 0.3841
Epoch [36/1000], Loss: 0.3000
Epoch [38/1000], Loss: 0.4744
Epoch [40/1000], Loss: 0.1362
Epoch [42/1000], Loss: 0.4877
Epoch [44/1000], Loss: 0.3204
Epoch [46/1000], Loss: 0.1074
patience exceeded, loading best model
Epoch [2/1000], Loss: 0.4091
Epoch [4/1000], Loss: 0.5096
Epoch [6/1000], Loss: 0.7925
Epoch [8/1000], Loss: 0.4755
Epoch [10/1000], Loss: 0.4511
Epoch [12/1000], Loss: 0.9101
Epoch [14/1000], Loss: 0.2763
Epoch [16/1000], Loss: 0.4879
Epoch [18/1000], Loss: 0.3763
Epoch [20/

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [2/1000], Loss: 0.5116
Epoch [4/1000], Loss: 0.7994
Epoch [6/1000], Loss: 0.4196
Epoch [8/1000], Loss: 1.1113
Epoch [10/1000], Loss: 0.4642
Epoch [12/1000], Loss: 0.1805
Epoch [14/1000], Loss: 0.5844
Epoch [16/1000], Loss: 0.2492
Epoch [18/1000], Loss: 0.4587
Epoch [20/1000], Loss: 0.2498
Epoch [22/1000], Loss: 0.4392
Epoch [24/1000], Loss: 0.3331
Epoch [26/1000], Loss: 0.2108
Epoch [28/1000], Loss: 0.6780
Epoch [30/1000], Loss: 0.1836
Epoch [32/1000], Loss: 0.2376
Epoch [34/1000], Loss: 0.9030
Epoch [36/1000], Loss: 0.1056
Epoch [38/1000], Loss: 0.0596
Epoch [40/1000], Loss: 0.3233
Epoch [42/1000], Loss: 0.5481
Epoch [44/1000], Loss: 0.1389
Epoch [46/1000], Loss: 0.2886
patience exceeded, loading best model


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [2/1000], Loss: 0.6488
Epoch [4/1000], Loss: 1.1505
Epoch [6/1000], Loss: 0.3241
Epoch [8/1000], Loss: 0.6017
Epoch [10/1000], Loss: 0.4283
Epoch [12/1000], Loss: 0.3300
Epoch [14/1000], Loss: 0.3866
Epoch [16/1000], Loss: 0.3612
Epoch [18/1000], Loss: 0.4281
Epoch [20/1000], Loss: 0.2075
Epoch [22/1000], Loss: 0.2869
Epoch [24/1000], Loss: 0.3165
Epoch [26/1000], Loss: 0.2032
Epoch [28/1000], Loss: 0.4240
Epoch [30/1000], Loss: 0.1334
Epoch [32/1000], Loss: 0.2129
Epoch [34/1000], Loss: 0.2508
Epoch [36/1000], Loss: 0.0953
Epoch [38/1000], Loss: 0.4492
Epoch [40/1000], Loss: 0.4914
Epoch [42/1000], Loss: 0.2871
Epoch [44/1000], Loss: 0.0601
Epoch [46/1000], Loss: 0.1580
patience exceeded, loading best model


...

Epoch [2/1000], Loss: 0.6757
Epoch [4/1000], Loss: 0.6698
Epoch [6/1000], Loss: 0.5100
Epoch [8/1000], Loss: 0.3324
Epoch [10/1000], Loss: 0.3998
Epoch [12/1000], Loss: 0.4334
Epoch [14/1000], Loss: 0.2219
Epoch [16/1000], Loss: 0.3984
Epoch [18/1000], Loss: 0.3496
Epoch [20/1000], Loss: 0.4544
Epoch [22/1000], Loss: 0.3443
Epoch [24/1000], Loss: 0.8499
Epoch [26/1000], Loss: 0.1666
Epoch [28/1000], Loss: 0.4261
Epoch [30/1000], Loss: 0.4885
Epoch [32/1000], Loss: 0.4484
Epoch [34/1000], Loss: 0.3061
Epoch [36/1000], Loss: 0.3348
Epoch [38/1000], Loss: 0.2513
Epoch [40/1000], Loss: 0.4365
Epoch [42/1000], Loss: 0.6335
Epoch [44/1000], Loss: 0.1946
Epoch [46/1000], Loss: 0.1610
Epoch [48/1000], Loss: 0.2587
patience exceeded, loading best model


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [2/1000], Loss: 0.3803
Epoch [4/1000], Loss: 0.4799
Epoch [6/1000], Loss: 0.9152
Epoch [8/1000], Loss: 0.3577
Epoch [10/1000], Loss: 0.9185
Epoch [12/1000], Loss: 0.1862
Epoch [14/1000], Loss: 0.2386
Epoch [16/1000], Loss: 0.3955
Epoch [18/1000], Loss: 0.3405
Epoch [20/1000], Loss: 0.3687
Epoch [22/1000], Loss: 0.3193
Epoch [24/1000], Loss: 0.3109
Epoch [26/1000], Loss: 0.1242
Epoch [28/1000], Loss: 0.2732
Epoch [30/1000], Loss: 0.2352
Epoch [32/1000], Loss: 0.2796
Epoch [34/1000], Loss: 0.0861
Epoch [36/1000], Loss: 0.2864
Epoch [38/1000], Loss: 0.3674
Epoch [40/1000], Loss: 0.3892
Epoch [42/1000], Loss: 0.1375
Epoch [44/1000], Loss: 0.4994
Epoch [46/1000], Loss: 0.2572
Epoch [48/1000], Loss: 0.1936
patience exceeded, loading best model


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [2/1000], Loss: 0.9900
Epoch [4/1000], Loss: 0.6932
Epoch [6/1000], Loss: 0.2789
Epoch [8/1000], Loss: 0.7136
Epoch [10/1000], Loss: 0.3626
Epoch [12/1000], Loss: 0.4433
Epoch [14/1000], Loss: 0.5696
Epoch [16/1000], Loss: 0.0825
Epoch [18/1000], Loss: 0.5120
Epoch [20/1000], Loss: 0.1993
Epoch [22/1000], Loss: 0.5736
Epoch [24/1000], Loss: 0.4961
Epoch [26/1000], Loss: 0.1299
Epoch [28/1000], Loss: 0.3444
Epoch [30/1000], Loss: 0.0677
Epoch [32/1000], Loss: 0.2165
Epoch [34/1000], Loss: 0.3713
Epoch [36/1000], Loss: 0.4412
Epoch [38/1000], Loss: 0.3011
Epoch [40/1000], Loss: 0.3397
Epoch [42/1000], Loss: 0.1863
Epoch [44/1000], Loss: 0.5806
Epoch [46/1000], Loss: 0.5157
patience exceeded, loading best model
Average accuracy: 0.8103964
Average top_k_average_accuracies 0.71312934
KNN accuracy: 0.9167228


# Results Interpretation

In [ ]:
def print_model_features(input_model):
  for n, p in model.named_parameters():
    print(n)
    print(p.data)

In [ ]:
print_model_features(model)

In [ ]:
# for regression only. for classification is different
#feature_activations, case_activations, predicted_number
model.eval()
feature_activations, case_activations, output, predicted_class = model(X_test)

In [ ]:
predicted_class

In [ ]:
y_test

In [ ]:
# prompt: accuracy comparing predicted_class and y_test

accuracy = accuracy_score(y_test.numpy(), predicted_class.cpu().numpy())
print("Accuracy:", accuracy)


In [ ]:
#inspecting the case activations
top_case_indices = torch.topk(case_activations, 5, dim=1)[1]

In [ ]:
X_test[0]

In [ ]:
y_test[0]

In [ ]:
X_train[top_case_indices[0][0]]

In [ ]:
y_train[top_case_indices[0][0]]

By comparing the following two blocks' outputs, you can see we are retrieving a good neighbor.

In [ ]:
#sum abs of X_test[0] and the top activated case
sum(abs(X_test[0] - X_train[top_case_indices[0][0]]))

In [ ]:
# prompt: average sum abs of X_test[0] and X_train data
print(np.mean([sum(abs(X_test[0] - X_train[i])) for i in range(len(X_train))]))

TODO:: A better way is to show the distribution of ``X_test[0] - X_train[i]``

In [ ]:
y_train[top_case_indices[0]]

In [ ]:
knn.predict(X_test)[0]

In [ ]:
indices = knn.kneighbors(X_test)[1][0]

In [ ]:
y_train[indices]

# Sanity Check


## Classification Neural Network

In [ ]:
# Hyperparameters
input_size = X_train.shape[1]
hidden_size = 1024
num_classes = torch.unique(ys).shape[0]
learning_rate = 1e-5
batch_size = 16
epochs = 2000

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define the neural network architecture for classification
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.nn = nn.Sequential(
            nn.Linear(input_size, hidden_size ),
            nn.LeakyReLU(),
            nn.Linear(hidden_size , hidden_size // 2),
            # nn.Dropout(0.5),
            nn.LeakyReLU(),
            nn.Linear(hidden_size // 2, hidden_size // 4),
            # nn.Dropout(0.5),
            nn.LeakyReLU(),
            nn.Linear(hidden_size // 4, num_classes)
            )
        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                torch.nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    m.bias.data.fill_(0)

    def forward(self, x):
        return self.nn(x)


In [ ]:
best_accuracies = []
for train_index, test_index in k_fold.split(Xs):
  train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
  patience_counter = 0
  best_model = None
  best_accuracy = None
  # Initialize the model, loss function, and optimizer
  model = NeuralNet(input_size, hidden_size, num_classes)
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

  # Training loop
  for epoch in range(epochs):
    epoch_msg = True
    training_total_acc = 0.0
    training_total_loss = 0.0
    num_of_batches = len(train_loader)
    for X_train_batch, y_train_batch in train_loader:
      model.train()
      # Forward pass
      outputs = model(X_train_batch)
      loss = criterion(outputs, y_train_batch)

      # Backward and optimize
      _, predicted = torch.max(outputs, 1)
      training_total_acc += torch.sum(predicted == y_train_batch).item()

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      training_total_loss += loss.item()
      # if (i + 1) % 5 == 0
    if epoch == 0 or (epoch + 1) % 100 == 0:
      print(f"Epoch: {epoch + 1}, Training Loss: {training_total_loss/num_of_batches:.2f} Acc: {training_total_acc/num_of_batches:.2f}")
    # Testing the model
    model.eval()
    with torch.no_grad():
      outputs = model(X_test)
      loss = criterion(outputs, y_test)
      _, predicted = torch.max(outputs, 1)
      accuracy = torch.sum(predicted == y_test).item() / len(y_test)
      print(f'Accuracy on the test set: {accuracy * 100:.2f}%')
      if best_accuracy is None or accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model
        patience_counter = 0
      else:
        patience_counter += 1
      if epoch_msg and (epoch + 1) % 100 == 0:
        epoch_msg = False
        print(f'Epoch [{epoch + 1}/{epoch}], Test Loss: {loss.item()}')
    if patience_counter >= cfg.patience:
      print("Best acc achieved: ", best_accuracy)
      break
  best_accuracies.append(best_accuracy)
print("Average accuracy:", np.mean(best_accuracies))

## Regression Neural Network

In [29]:
# Hyperparameters
input_size = X_train.shape[1]
hidden_size = 100
# num_classes = torch.unique(ys).shape[0]
learning_rate = 1e-5
batch_size = 16
epochs = 2000

In [30]:
# prompt: a standard neural network with 3 fully connected layers for regression

import torch
import torch.nn as nn
import torch.nn.functional as F

class RegressionNet(nn.Module):
    def __init__(self, input_size):
        super(RegressionNet, self).__init__()
        self.nn = nn.Sequential(
            nn.Linear(input_size, hidden_size ),
            nn.LeakyReLU(),
            nn.Linear(hidden_size , hidden_size // 2),
            # nn.Dropout(0.5),
            nn.LeakyReLU(),
            nn.Linear(hidden_size // 2, hidden_size // 4),
            # nn.Dropout(0.5),
            nn.LeakyReLU(),
            nn.Linear(hidden_size // 4, 1)
            )
        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                torch.nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    m.bias.data.fill_(0)

    def forward(self, x):
        return self.nn(x).squeeze()

In [32]:
best_accuracies = []
for train_index, test_index in k_fold.split(Xs):
  # Get training and testing data
  X_train, X_test = Xs[train_index], Xs[test_index]
  y_train, y_test = ys[train_index], ys[test_index]
  train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
  patience_counter = 0
  best_model = None
  best_accuracy = None
  model = RegressionNet(Xs.shape[1])
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  for epoch in range(epochs):
    epoch_msg = True
    training_total_loss = 0.0
    num_of_batches = len(train_loader)
    for X_train_batch, y_train_batch in train_loader:
      model.train()
      # Forward pass
      outputs = model(X_train_batch)
      loss = criterion(outputs, y_train_batch)
      training_total_loss += loss.item()
      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    if epoch == 0 or (epoch + 1) % 3 == 0:
      print(f'Epoch: {epoch + 1}, Training Loss: {training_total_loss/num_of_batches:.2f}')

    # Testing the model
    model.eval()
    with torch.no_grad():
      outputs = model(X_test)
      loss = criterion(outputs, y_test)
      if best_accuracy is None or loss.item() < best_accuracy:
        best_accuracy = loss.item()
        best_model = model
        patience_counter = 0
      else:
        patience_counter += 1
      if epoch_msg and (epoch + 1) % 100 == 0:
        epoch_msg = False
        print(f'Epoch [{epoch + 1}/{epochs}], Test Loss: {loss.item()}')
        # print(f'Loss on the test set: {loss.item()}')
    if patience_counter >= cfg.patience:
      print("Best loss achieved: ", best_accuracy)
      break
  best_accuracies.append(best_accuracy)
print("Average accuracy:", np.mean(best_accuracies))


Epoch: 1, Training Loss: 1.11
Epoch: 3, Training Loss: 1.00
Epoch: 6, Training Loss: 0.97
Epoch: 9, Training Loss: 0.96
Epoch: 12, Training Loss: 0.95
Epoch: 15, Training Loss: 0.93
Epoch: 18, Training Loss: 0.92
Epoch: 21, Training Loss: 0.91
Epoch: 24, Training Loss: 0.90
Epoch: 27, Training Loss: 0.89
Epoch: 30, Training Loss: 0.88
Epoch: 33, Training Loss: 0.87
Epoch: 36, Training Loss: 0.85
Epoch: 39, Training Loss: 0.84
Epoch: 42, Training Loss: 0.83
Epoch: 45, Training Loss: 0.81
Epoch: 48, Training Loss: 0.80
Epoch: 51, Training Loss: 0.79
Epoch: 54, Training Loss: 0.77
Epoch: 57, Training Loss: 0.76
Epoch: 60, Training Loss: 0.74
Epoch: 63, Training Loss: 0.72
Epoch: 66, Training Loss: 0.71
Epoch: 69, Training Loss: 0.70
Epoch: 72, Training Loss: 0.69
Epoch: 75, Training Loss: 0.67
Epoch: 78, Training Loss: 0.66
Epoch: 81, Training Loss: 0.65
Epoch: 84, Training Loss: 0.64
Epoch: 87, Training Loss: 0.63
Epoch: 90, Training Loss: 0.63
Epoch: 93, Training Loss: 0.62
Epoch: 96, T

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 1, Training Loss: 1.10
Epoch: 3, Training Loss: 1.01
Epoch: 6, Training Loss: 0.98
Epoch: 9, Training Loss: 0.97
Epoch: 12, Training Loss: 0.96
Epoch: 15, Training Loss: 0.94
Epoch: 18, Training Loss: 0.93
Epoch: 21, Training Loss: 0.90
Epoch: 24, Training Loss: 0.89
Epoch: 27, Training Loss: 0.88
Epoch: 30, Training Loss: 0.85
Epoch: 33, Training Loss: 0.83
Epoch: 36, Training Loss: 0.81
Epoch: 39, Training Loss: 0.79
Epoch: 42, Training Loss: 0.79
Epoch: 45, Training Loss: 0.76
Epoch: 48, Training Loss: 0.74
Epoch: 51, Training Loss: 0.73
Epoch: 54, Training Loss: 0.71
Epoch: 57, Training Loss: 0.70
Epoch: 60, Training Loss: 0.69
Epoch: 63, Training Loss: 0.68
Epoch: 66, Training Loss: 0.67
Epoch: 69, Training Loss: 0.65
Epoch: 72, Training Loss: 0.65
Epoch: 75, Training Loss: 0.63
Epoch: 78, Training Loss: 0.63
Epoch: 81, Training Loss: 0.62
Epoch: 84, Training Loss: 0.61
Epoch: 87, Training Loss: 0.61
Epoch: 90, Training Loss: 0.60
Epoch: 93, Training Loss: 0.62
Epoch: 96, T